In [1]:
import requests
import numpy as np
import os

In [2]:
texts = [os.path.join('./lectures', text) for text in os.listdir('./lectures')]
g = open('essay.txt', 'w')
for text in texts:
    with open(text, "r",  encoding="utf-8") as f:
        t = f.read()
        g.write(t)
g.close()

In [3]:
with open('essay.txt', "r") as g:
    text = g.read()
    chunk_size = 2048
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [4]:
len(chunks)

37

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(api_key='',model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_index = FAISS.from_texts(chunks, embedding=embeddings)

In [9]:
retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [27]:
from transformers import pipeline

hf_pipeline = pipeline(
    "text-generation",
    model="meta-llama/llama-3.2-1B",
    token=''
)

model.safetensors:   6%|5         | 147M/2.47G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cpu


In [28]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=hf_pipeline)

C:\Users\asche\AppData\Local\Temp\ipykernel_19208\2527154434.py:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [29]:
import langchain.chains
import langchain.prompts

prompt = """
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста. Выведи только
краткий ответ, не надо пояснительного текста.
Текст:
-----
{context}
-----
Вопрос:
{question}
"""

prompt = langchain.prompts.PromptTemplate(
    template=prompt, input_variables=["context", "question"]
)

chain = langchain.chains.RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
)

In [31]:
from IPython.display import display, Markdown, Math

In [60]:
query = "Что такое условная вероятность?"
result = chain.run(query)
print(result)

Условной вероятностью события $A$ при условии наступления события $B$ называется величина $ P(A | B) = rac{P(A \cap B)}{P(B)} $
